In [195]:
import pandas as pd
import json
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [196]:
with open("lyrics.json", 'r') as f:
    song_lyrics = json.load(f)

In [197]:
song_lyrics = [lyrics for lyrics in song_lyrics if lyrics != "None"]

In [198]:
for i in range(len(song_lyrics)):
    song_lyrics[i] = re.sub("([a-z])([A-Z])", " \\1", str(song_lyrics[i])).strip()
    song_lyrics[i] = re.sub("[\(\[].*?[\)\]]", "", str(song_lyrics[i]))
    song_lyrics[i] = re.sub(r"\s*{.*}\s*", " ", str(song_lyrics[i]))
    song_lyrics[i] = song_lyrics[i].replace("\n", " ")
    song_lyrics[i] = re.sub(' +', ' ', str(song_lyrics[i]))

In [199]:
len(song_lyrics)

66

In [200]:
tokenizer = RegexpTokenizer("\w+|\$[\d\.]+|\S+")

In [201]:
stop_words = list(set(stopwords.words("english")))
stop_words += list(string.punctuation)

In [202]:
stop_words

['yourself',
 'should',
 'about',
 'do',
 'until',
 'up',
 'shouldn',
 'while',
 'than',
 'had',
 'no',
 'in',
 'that',
 'did',
 'each',
 'are',
 'but',
 'after',
 "you've",
 'haven',
 'will',
 'our',
 'they',
 'been',
 'aren',
 'here',
 'those',
 "hasn't",
 'out',
 'was',
 'with',
 'myself',
 'the',
 'just',
 "wouldn't",
 'y',
 's',
 'any',
 'if',
 'ours',
 'once',
 'ma',
 'which',
 'by',
 'it',
 'for',
 "won't",
 'is',
 're',
 'there',
 'couldn',
 'during',
 'hasn',
 'didn',
 'he',
 'too',
 'we',
 'herself',
 'its',
 'i',
 'll',
 'off',
 'your',
 't',
 'what',
 'them',
 'when',
 'does',
 'm',
 "didn't",
 'whom',
 'an',
 "weren't",
 "mustn't",
 'hers',
 'same',
 "aren't",
 'this',
 'being',
 'won',
 'all',
 'doesn',
 "haven't",
 "mightn't",
 'a',
 'who',
 'wouldn',
 "don't",
 'before',
 'under',
 'has',
 'don',
 "shan't",
 'to',
 'further',
 'or',
 "doesn't",
 "it's",
 'where',
 'more',
 'am',
 "should've",
 'as',
 'yours',
 'then',
 'my',
 'so',
 'ain',
 'now',
 'how',
 'again',
 'ot

In [203]:
sample_tfidf = TfidfVectorizer(lowercase=True,
                              stop_words=stop_words,
                              tokenizer=tokenizer.tokenize,
                              min_df=1)

In [208]:
sample_sparse = sample_tfidf.fit_transform(song_lyrics)

In [209]:
sample_exp = pd.DataFrame(sample_sparse.toarray(), 
                        columns=sample_tfidf.get_feature_names())

sample_exp.head()

,"""pay""","',",'18,'?,'ave,'bout,'cause,'chieve,'cuh,'d,...,‘,’bout,’ll,’m,’re,’s,"’s,",’t,’ve,…
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040331,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.134836,0.0,0.0,0.361554,0.0,0.0,0.0,0.144995
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [210]:
sample_exp.shape

(66, 2242)

In [211]:
tfidf = TfidfVectorizer(lowercase=True, 
                        stop_words=stop_words, 
                        tokenizer=tokenizer.tokenize,
                        min_df=0.05)
tfidf_train_sparse = tfidf.fit_transform(song_lyrics)
tfidf_train_df = pd.DataFrame(tfidf_train_sparse.toarray(), 
                        columns=tfidf.get_feature_names())

In [212]:
tfidf_train_df.head()

,"',",'bout,'cause,'d,'em,'ll,'m,'re,'round,'s,...,would,wrong,ya,yeah,year,years,young,’m,’s,’t
0,0.0,0.0,0.0,0.078556,0.0,0.077216,0.077025,0.000000,0.000000,0.037911,...,0.139751,0.0,0.000000,0.064893,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.035414,0.283724,0.000000,0.000000,...,0.000000,0.0,0.083868,0.134262,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.225136,0.042777,0.000000,0.000000,0.147379,...,0.077612,0.0,0.000000,0.027029,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.472091,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.000000,0.059153,0.000000,0.146713,0.058228,...,0.000000,0.0,0.000000,0.074753,0.0,0.0,0.0,0.0,0.000000,0.0
